# Setup

### Path setup

In [1]:
import os

In [2]:
path_to_this_folder = os.getcwd() # Do it before calling %cd ../.. so that the path is accurate

### Project setup

In [3]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"   # Prevent internal ngsolve imports from killing the kernel


In [4]:
%cd ../..
!pip install .
!pip install meshio
!pip install ngsolve




c:\Users\octav\Documents\GitHub\Stage_ARIA
Processing c:\users\octav\documents\github\stage_aria
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for yourpkg: filename=yourpkg-0.1.0-py3-none-any.whl size=32722 sha256=96503f02f46bff24e1fd52c1f16042dce8672bd045aee99d20c929cad102c093
  Stored in directory: C:\Users\octav\AppData\Local\Temp\pip-ephem-wheel-cache-1srgohxx\wheels\2f\97\6f\4cd56f403f3905a3673325538741a3519ba70ae59cdd7d492e
Successfully built yourpkg
  Attempting uninstall: yourpkg
    Found existing installation: yourpkg 0.1.0
    Uninstalling yourpkg-0.1.0:
      Successfully uninstalled yourpkg-0.1.0


### Imports

Usual libraries

In [5]:
import tomllib
from time import time
import torch
from netgen.read_gmsh import ReadGmsh
from ngsolve import Mesh, H1, GridFunction, Integrate, dx, grad, BilinearForm, LinearForm
import matplotlib.pyplot as plt

Custom code 

In [6]:
from src.Pre_processing.process_hardware import get_precision, get_device
from src.Pre_processing.process_gmsh import read_gmsh, get_nodes_and_elements_IDs
from src.FENN.VertexNN.FEENN_2D.FEENN import FEENN_2D
from src.FENN.VertexNN.FEENN_2D.Element import Tri_2D_lin
from src.FENN.VertexNN.FEENN_2D.Mapping import Mapping_2D_Affine
from src.FENN.ConstantNN.constantNN import ConstantNN
from src.PropertiesNN.PropertiesNN import PropertiesNN
from src.Pre_processing.build_solver import build_solver
from src.PDE.poisson import Poisson_2D
from src.Post_processing.VTKExport.export_vtk import export_vtk
from src.Baselines.Poisson_2D.Uniform.baseline import baseline


### Number precision

In [7]:
IntPrecision = torch.int32
FloatPrecision = torch.float64

### Hardware

Note that for cases of small meshes cpu outperforms gpu

In [8]:
device = 'cpu'

# Problem description and problem-related variables definition

$$
\begin{align*}
\;&\text{Find}&&V^*&&&=&&&&\arg\min_{V\in H_1(\Omega)} &&&&& \int_\Omega \epsilon\cdot \nabla V\cdot \nabla V-\rho\cdot V d\Omega\\\\
&\text{s.t}&&u(x) &&&=&&&&0&&&&&\forall x\in\partial\Omega\\
&&&\epsilon(x)&&&=&&&&\text{cst}&&&&&\forall x\in\Omega\\
&&&\rho(x)&&&=&&&&\text{cst}&&&&&\forall x\in\Omega

\end{align*}
$$

$\Omega$ is defined through a .msh file

In [9]:
path_to_msh = os.path.join(path_to_this_folder, "Poisson_2D_uniform.msh")

The material is constant (arbitrarily set to 1)

In [10]:
epsilon_value = 1.0


The source is constant

In [11]:
rho_value = 1.0

The problem is set in 2D

In [12]:
interior_dim = 2
boundary_dim = 1

$V$ is a scalar field that is null on the boundary. Note that the notion of boundary is gmsh-dependent. In other words, the dirichlet config should be consistant with the Physical Entities of your .msh file.

In [13]:
n_components = 1

dirichlet_bottom = {'Entity'    :   111,
                    'name'      :   'Dirichlet_bottom',
                    'Value'     :   0}

dirichlet_right = {'Entity'    :   112,
                    'name'      :   'Dirichlet_right',
                    'Value'     :   0}

dirichlet_top = {'Entity'    :   113,
                    'name'      :   'Dirichlet_top',
                    'Value'     :   0}

dirichlet_left = {'Entity'    :   114,
                    'name'      :   'Dirichlet_left',
                    'Value'     :   0}

dirichlet_config = [dirichlet_bottom, dirichlet_right, dirichlet_top, dirichlet_left]

We use a linear interpolation for our FEM representation of $V$. Therefore $\nabla V$ is constant by element and the appropriate integration scheme is an order 1 gaussian quadrature.

In [14]:
quadrature_order = 1

# Baseline with NGSolve

Solve the problem with NGSolve to get a baseline.

Fore more info, see : https://docu.ngsolve.org/latest/

In [15]:
# Extract the mesh-related info from gmsh
mesh = Mesh(ReadGmsh(path_to_msh))
boundaries = mesh.GetBoundaries()

# Set the problem 
dirichlet = ''
for dirichlet_subconfig in dirichlet_config:
    if dirichlet_subconfig['name'] in boundaries:
        dirichlet += f"{dirichlet_subconfig['name']}|"

fes = H1(mesh, order = 1, dirichlet = dirichlet)

u = fes.TrialFunction()
v = fes.TestFunction()




dirichlet = dirichlet[:-1]

bf = BilinearForm(fes)
bf += (epsilon_value*grad(u)*grad(v))*dx

lf = LinearForm(fes)
lf += rho_value*v*dx

bf.Assemble()
lf.Assemble()

# Solve
sol = GridFunction(fes)
sol.vec.data = bf.mat.Inverse(fes.FreeDofs(), inverse = "sparsecholesky")*lf.vec

# Get energy
energy_source = Integrate((rho_value*sol)*dx, mesh)
energy_field = Integrate((0.5 * epsilon_value *  grad(sol)*grad(sol))*dx, mesh)
energy = energy_field - energy_source

# FENN solve

Read the gmsh file and get the mesh related info

In [16]:
gmsh_mesh = read_gmsh(path_to_msh = path_to_msh,
                      IntPrecision = IntPrecision,
                      FloatPrecision = FloatPrecision)


Nodes = gmsh_mesh.nodes
connectivity = gmsh_mesh.elements[str(interior_dim)]['connectivity']

Processing GMSH file: 100%|██████████| 214/214 [00:00<00:00, 61184.80it/s]


Build a field candidate

In [17]:
# Build the field candidate. Here we use a linear interpolation scheme.
V = FEENN_2D(Nodes = Nodes,
             connectivity=connectivity,
             n_components=n_components,
             element=Tri_2D_lin(IntPrecision=IntPrecision, FloatPrecision=FloatPrecision),
             mapping = Mapping_2D_Affine(),
             IntPrecision=IntPrecision,
             FloatPrecision=FloatPrecision)

# Apply dirichlet
for dirichlet in dirichlet_config:

    nodeIDs, _ = get_nodes_and_elements_IDs(gmsh_mesh=gmsh_mesh,
                                                  entity_dim=boundary_dim,
                                                  entity_tag=dirichlet['Entity'])
    
    
    value = dirichlet['Value']

    V.SetBCs(Fixed_nodal_coordinates_Ids = nodeIDs,
             Fixed_nodal_values_Ids = nodeIDs,
             Fixed_nodal_values_values = value)
    

# Specify the integration scheme
V.SetQuad(quadrature_order = quadrature_order)
V.Freeze(freeze_grid=True, freeze_interpolation=False)

# Move the object to the chosen hardware
V.to(device)


c:\Users\octav\Documents\GitHub\Stage_ARIA\src\FENN\VertexNN\FEENN_2D\FEENN.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('connectivity', torch.tensor(connectivity, dtype = self.ref_int.dtype))


FEENN_2D(
  (grid): GridNN_2D(
    (nodal_coordinates): ParameterDict(
        (free): Parameter containing: [torch.DoubleTensor of size 41x3]
        (imposed): Parameter containing: [torch.DoubleTensor of size 24x3]
    )
  )
  (interpolation): InterpolationNN_2D(
    (nodal_values): ParameterDict(
        (free): Parameter containing: [torch.DoubleTensor of size 41x1]
        (imposed): Parameter containing: [torch.DoubleTensor of size 24x1]
    )
    (element): Tri_2D_lin()
    (mapping): Mapping_2D_Affine()
  )
)

Define the material

In [ ]:
# Define the epsilon property
epsilon = ConstantNN(property_value=epsilon_value,
                     IntPrecision = IntPrecision,
                     FloatPrecision = FloatPrecision)



# epsilon is not a field to be found 
epsilon.setBCs(is_fixed=True)

# The only problem-relevant material property is epsilon
Mat = PropertiesNN(dim = interior_dim,
                 NElem = len(connectivity),
                 IntPrecision = IntPrecision,
                 FloatPrecision = FloatPrecision)

Mat.add_property(property_name = 'epsilon',
                 property = epsilon)

# Move the object to the chosen hardware
# Mat.to(device)

PropertiesNN(
  (properties): ModuleDict(
    (epsilon): ConstantNN()
  )
)

Define the source

In [19]:
# Define the source term
rho = ConstantNN(property_value = rho_value,
                 IntPrecision = IntPrecision,
                 FloatPrecision = FloatPrecision)

# The source is not a field to be found 
rho.setBCs(is_fixed=True)

# Move the object to the chosen hardware
rho.to(device)

ConstantNN()

Define the energy functional and how the field of interest should be saved

In [20]:
vtk_export_config = {'path_to_folder'       : os.path.join(os.path.join(path_to_this_folder, 'Results'), 'VTK_exports'),
                    'export_vtk'            : True}

In [21]:
energy_functionnal = Poisson_2D(vtk_export=vtk_export_config, 
                                baseline = sol, 
                                baseline_mesh=mesh)

Specify the solver and build it. Here, the objective is convex so LBFGS (quasi-Newton) is appropriate.

In [22]:
solver_config = {'optimizer'               :   'lbfgs',
                'n_epochs'                :   500,
                'lr'                      :   1.0e-4,
                'loss_decrease_c'         :   1e-13,
                'freeze_grid'             :   True,
                'freeze_values'           :   False,
                'freeze_Mat'              :   True}

In [23]:
solver = build_solver(solver_config = solver_config,
                      loss = energy_functionnal,
                      model = V,
                      Mat = Mat,
                      source = rho)

Solving (worse case scenario):   0%|          | 0/500 [00:00<?, ?it/s]

Solve

In [24]:
solver.solve()

Solving (worse case scenario):   0%|          | 1/500 [00:00<00:10, 49.15it/s, time=0]c:\Users\octav\Documents\GitHub\Stage_ARIA\src\PDE\poisson.py:81: RuntimeWarning: invalid value encountered in divide
  relat_error = abs(exact_values - nodal_values)/abs(exact_values)
Solving (worse case scenario):   1%|          | 3/500 [00:00<00:15, 32.09it/s, time=0.0696]

See the results in the VTK file